# Running a Regression

We will be running a lot of regressions and related models in this course.  This cheat sheet will show you how to run a regression on the sales data tab of the Class 3 Problem set.  As you will see, aside from some initial challenges with figuring out the command’s syntax, running a regression is generally very easy.  Knowing what to run and what to do with the results – that is much more challenging.

We will start this set of Cheat Sheet tasks by building a very simple linear regression model:

$$
\text{Order_Size} = \beta_0 + \beta_1 \text{Ad_Budget} + \beta_2 \text{Dist}
$$

## Instructions

**1.	Start a new project and load the sales data**

This data is tab 2 in "MMA 860 Assessing and Testing Data File v1.0". 

In [ ]:
#Import the required libraries
import pandas as pd
import os.path as osp

#Build the path for the data file
data_path = osp.join(
    osp.curdir,'Data','MMA 860 Assessing and Testing Data File v1.0.xlsx')

#Use the read_excel function to pull data from the 'Sales Data' sheet
data = pd.read_excel(
    data_path,sheet_name='Sales Data',index_col='Observation')
data.head()

**2. Run some summary statistics**

As before, running $\text{dtypes}$ and $\text{describe()}$ is a good way to get a summary of your dataset.

In [ ]:
data.dtypes

In [ ]:
data.describe()

**3. Look at the linear regression help file**

To create our model, we will use the sci-kit learn library. Referencing the following documentation for linear regression, we can train our model.

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
#Import the required package from sklearn
from sklearn.linear_model import LinearRegression

'''
In order to input data from a pandas dataframe and into a sci-kit function,
we need to convert the dataframe series into NumPy Arrays. This can be done
with the values function.
'''
train_X = data[['Ad_Budget','Distance']].values
train_y = data['Order_Size'].values

'''
Fitting data to a regression model requires two arguments, the training X
values (independent variables) and the training y values (dependent variables.
In general, most fit functions for models follow this format.
'''
reg = LinearRegression().fit(train_X, train_y)

You’ve now trained your first predictive model! Before proceeding, explore the link above to look at ways to assess your models performance. Additionally, test out the functions used to print the coefficients.

# Assessing a Model

I think of model assessment as following this process:

1. Does the model have any predictive power? (Check the F-Test)
2. Do the variables we have included belong in the model? (Check the T-Tests)
3. Have we violated any regression assumptions (Check the plots)
4. Is the $R^2$ sufficient for business requirements?

$R^2$ is an assessment of model fit – it represents the proportion of variability in $y$ explained by the model. The **Adjusted** $R^2$ adjusts for the number of variables included in a model, and is a better measure of model fit. The following script outputs the $R^2$ of the model:

In [ ]:
print("R-Squared:", reg.score(train_X, train_y))

This linear regression model is defined by the three $\beta$ coefficients seen above - $\beta_0$ is sometimes called the intercept. The following script produces the coefficients for each variable, which are used to create predictions:

In [ ]:
'''
Computing intercept is trivial. The 'slope' coefficients are outputted as a
tuple (consider this analogous to an array or list). We must index the tuple
to get the numeric values. The values appear in the tuple in an order that 
corresponds to the position of the independent variable to which they are the
slope of. For example, X_train has Ad_Budget values first then Distance values 
which results in the order presented below.
'''

print("B_0 =",reg.intercept_)

#Ad_Budget Coefficient
print("B_1 =",reg.coef_[0])

#Distance Coefficient
print("B_2 =",reg.coef_[1])

The coefficients substitute for beta values in the regression equation. In this example, you would replace $\beta_0$ with $26.89$, $\beta_1$ with $0.0027$, and $\beta_2$ with $.144$. Then, by plugging in the values of any observation, you can produce a point estimate. Using the equation we developed, what is the predicted order size if $\text{Ad_Budget}$ is $1000$ and $\text{Dist}$ is $50$? As a bonus, use $\text{reg.predict()}$ to determine this value.

In [ ]:
import numpy as np

#The prediction function takes a Numpy array as an argument
reg.predict(np.array([1000,50]).reshape(1,-1))[0]

The **Joint F-Test** is a formal hypothesis test to assess whether a model has any predictive ability. It tests the null hypothesis that all coefficients are jointly equal to $0$.

The **t-tests** are formal hypothesis tests for each variable in the model. Each test assesses whether the coefficient for an individual variable is equal to $0$. 

### Stats Model

For linear regression there is an alternative library that exists called Statsmodel that can be used to do model assessment - yes it is in the Anaconda Distribution. One big advantage to using this library is the great summary that you get which contains the T and F statistics.

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

'''
Fitting a model involves passing two arguments to ols: the general formula as
a string and the data set used. Remember that the formula's attributes must
match the column names in the dataframe. Then the fit() function is run and
then summary() can be applied to that model.
'''
model = ols('Order_Size ~ Ad_Budget + Distance',data).fit()
print(model.summary())

## Plotting Regression Models 

In the next section, we will come across a measurement known as a **residual**. Residuals are the set of differences between our training data and our model's prediction.

A big indicator in the efficacy of the regression model is whether or not the residuals follow a **Normal Distribution**. This would suggest that the errors are due to random error and not some systematic flaw in our model. There are a few plots we can use to check this.

### Residual vs Fitted Values

The data plotted here should have no pattern (e.g., parabola, two clusters, cone shaped) and be evenly distributed around $0$ on the y-axis.

In [ ]:
#Residuals calculated by definition above.
predicted_y = reg.predict(train_X)
#Note we can perform element-wise subtraction between arrays like so
residuals = train_y - predicted_y

import matplotlib.pyplot as plt
plt.scatter(predicted_y,residuals,s=2,c='black')

#This line adds the dashed horizontal line
plt.hlines(0,min(predicted_y),max(predicted_y),color='red',linestyles='dashed')

plt.xlabel("Model Prediction")
plt.ylabel("Residual")
plt.show()

We can additionally look at how close the mean of the residuals is to zero.

In [ ]:
print('Mean of Residuals:',residuals.mean())

### Normal Q-Q

The Q-Q plot gives insight into **multivariate normality**. The plot should look similar to the one below – the further the points are from the diagonal line, the more likely there are normality problems.

In [ ]:
import scipy.stats as stats

'''
Boilerplate code for creating a Normal Q-Q plot. The first two lines declare 
a figure and a subplot. This is an alternate way to output plots which allows 
for more than one plot per output.
'''
fig = plt.figure()
ax = fig.add_subplot(111)

'''
Scipy.Stats has a built-in function for generating this type of plot. This 
function takes three arguments: the measurement being checked (residuals), 
the distribution we are checking against (normal in this case), and the plot 
to plot it to.
'''
stats.probplot(residuals,dist='norm',plot=ax)
plt.show()

### Scale-Location

Use this plot to check the assumptions of equal variance in errors (**homoscedasticity**). You want to see a horizontal line with points spread randomly on either side. If there is a clear cone shaped pattern, it is evidence of heteroskedasticity.

This plot has the predicted values on the x axis and the square root of the absolute value of the standardized residuals on the y axis. When a set of variables become standardized this means that they are referred to by the number of standard deviations that they lie away from their mean.

In [ ]:
from sklearn.preprocessing import StandardScaler

# First we create an array of normalized residuals using a Scikit function
scaler = StandardScaler().fit(residuals.reshape(-1,1))
norm_residuals = scaler.transform(residuals.reshape(-1,1))

# Plot and take the root and absolute values of the norms
plt.scatter(predicted_y,np.sqrt(np.abs(norm_residuals)),c='black',s=2)
plt.xlabel("Fitted Values")
plt.ylabel("Root of standardized residual")
plt.show()

### Density Plot

There is one last plot worth observing – the density plot. This is another plot used to check for **normality of errors**. You would like to see a largely normal distribution – any other distinct pattern is indicative of a violation in assumptions. You can view it with the following code:

In [ ]:
from scipy.stats import norm

# Fit a normal distribution to the data:
mean, std = norm.fit(residuals)

# Plot the histogram.
plt.hist(residuals, bins=13, edgecolor='black', density=True)

# Generate a PDF based on the fitted distribution
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mean, std)
plt.plot(x, p, color='black')
title = "Fit results: mu = %.2f,  std = %.2f" % (mean, std)
plt.title(title)

plt.show()

Take this moment to play around with the number of bins in the histogram. Note some observations on what happens if you have too many or too little bins.

### Residuals vs. Leverage 

The Cook’s Distance plot helps you identify observations with **high leverage** (i.e., those that greatly influence the regression results). In this plot, the x axis is leverage, the y axis is the Studentized residuals (this is similar to standardized residual) and the size of the points indicates the Cook's Distance. Such a plot can be generated with the $\text{influence_plot}$ function from the **StatsModel** library. We use the previously trained model in the code below.

In [ ]:
#In a similar fashion to how the QQ plot was built, this plot can be generated.
fig, ax = plt.subplots(figsize=(12,8))
fig = sm.graphics.influence_plot(model, ax=ax, criterion="cooks")
plt.show()